# Kymograph Analysis from NWB Dendritic Excitability Data

This notebook reproduces the kymograph figure from the dendritic excitability experiments using NWB files.
We'll extract the fluorescence imaging data and recreate the same analysis as shown in the original code.

In [ ]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBHDF5IO

warnings.filterwarnings('ignore')

# Set up matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Load NWB File

Let's load one of the dendritic excitability NWB files and explore its structure.

In [ ]:
# Find repository root and define path to NWB files
repo_root = Path("../")
assert (repo_root / "pyproject.toml").exists(), "Could not find repository root with pyproject.toml"

nwb_files_dir = repo_root / "nwb_files" / "figure_1_dendritic_excitability"
print(f"NWB files directory: {nwb_files_dir}")
print(f"Directory exists: {nwb_files_dir.exists()}")

# Get a list of available NWB files
if nwb_files_dir.exists():
    nwb_files = list(nwb_files_dir.glob("*.nwb"))
    print(f"Found {len(nwb_files)} NWB files")
    for i, file in enumerate(nwb_files[:5]):  # Show first 5
        print(f"{i}: {file.name}")
    
    # Select a file to analyze (choose one that matches the original example if possible)
    # Looking for a file that might contain the 0706a session mentioned in the original code
    selected_file = None
    for file in nwb_files:
        if "0706" in file.name or "LID_on-state" in file.name:
            selected_file = file
            break
    
    if selected_file is None and len(nwb_files) > 0:
        selected_file = nwb_files[0]  # Fall back to first file
    
    if selected_file:
        print(f"\nSelected file: {selected_file.name}")
    else:
        print("\nNo NWB files found in directory")
else:
    print("Directory does not exist. Please run the figure_1_dendritic_excitability.py conversion script first.")
    nwb_files = []
    selected_file = None

In [ ]:
# Load the NWB file
with NWBHDF5IO(selected_file, 'r') as io:
    nwbfile = io.read()
    
    print(f"Session: {nwbfile.session_description}")
    print(f"Experiment: {nwbfile.experiment_description}")
    print(f"Session ID: {nwbfile.session_id}")
    print(f"Session start time: {nwbfile.session_start_time}")
    
    print("\n=== Acquisition data ===")
    for name, data in nwbfile.acquisition.items():
        print(f"{name}: {type(data).__name__}")
        if hasattr(data, 'data'):
            print(f"  Shape: {data.data.shape if hasattr(data.data, 'shape') else 'N/A'}")
        if hasattr(data, 'description'):
            print(f"  Description: {data.description}")
    
    print("\n=== Processing modules ===")
    for name, module in nwbfile.processing.items():
        print(f"{name}: {type(module).__name__}")
        for container_name, container in module.data_interfaces.items():
            print(f"  {container_name}: {type(container).__name__}")
            if hasattr(container, 'data'):
                print(f"    Shape: {container.data.shape if hasattr(container.data, 'shape') else 'N/A'}")


## Extract Imaging Data

Let's look for fluorescence imaging data that would contain the kymograph information.

In [ ]:
# Reload and extract imaging data
with NWBHDF5IO(selected_file, 'r') as io:
    nwbfile = io.read()
    
    # Look for imaging data in processing modules
    imaging_data = None
    imaging_metadata = None
    
    # Check ophys module for fluorescence data
    if 'ophys' in nwbfile.processing:
        ophys_module = nwbfile.processing['ophys']
        print("Found ophys module with:")
        
        for name, container in ophys_module.data_interfaces.items():
            print(f"  {name}: {type(container).__name__}")
            
            # Look for fluorescence traces or ROI response series
            if 'Fluorescence' in type(container).__name__:
                print(f"    Found fluorescence data: {name}")
                if hasattr(container, 'roi_response_series'):
                    for roi_name, roi_data in container.roi_response_series.items():
                        print(f"      ROI: {roi_name}, Shape: {roi_data.data.shape}")
                        if imaging_data is None:  # Take the first one we find
                            imaging_data = roi_data.data[:]
                            imaging_metadata = {
                                'name': roi_name,
                                'description': roi_data.description if hasattr(roi_data, 'description') else 'N/A',
                                'unit': roi_data.unit if hasattr(roi_data, 'unit') else 'N/A'
                            }
    
    # Also check for raw imaging data in acquisition
    for name, data in nwbfile.acquisition.items():
        if 'TwoPhoton' in type(data).__name__ or 'ImageSeries' in type(data).__name__:
            print(f"Found imaging data in acquisition: {name}")
            print(f"  Type: {type(data).__name__}")
            if hasattr(data, 'data'):
                print(f"  Shape: {data.data.shape}")
                if imaging_data is None:  # Take the first imaging data we find
                    raw_imaging = data.data[:]
                    print(f"  Loaded raw imaging data with shape: {raw_imaging.shape}")
    
    if imaging_data is not None:
        print(f"\nExtracted imaging data:")
        print(f"  Shape: {imaging_data.shape}")
        print(f"  Metadata: {imaging_metadata}")
    else:
        print("\nNo suitable imaging data found for kymograph analysis.")
        print("This NWB file may contain only electrophysiology data.")

## Line Scan Overlay Visualization

Let's create a visualization showing where the line scans are performed on the source image, similar to how the original analysis defines the scan line position.

In [ ]:
# Extract source image and line scan data from NWB file
source_image = None
kymograph_data = None
line_scan_coords = None
metadata = {}

if selected_file and selected_file.exists():
    with NWBHDF5IO(selected_file, 'r') as io:
        nwbfile = io.read()
        
        print("=== Extracting data from NWB file ===")
        
        # 1. Extract source image
        source_images_container = nwbfile.acquisition['SourceImagesLineScan']
        available_images = list(source_images_container.images.keys())
        print(f"Source images available: {available_images}")
        
        # Choose the first available Alexa568 image (structural reference)
        alexa_images = [img for img in available_images if 'Alexa568' in img]
        if alexa_images:
            source_image_name = alexa_images[0]  # Take the first one
            source_image = source_images_container.images[source_image_name].data[:]
            print(f"✓ Extracted source image: {source_image_name}")
            print(f"  Shape: {source_image.shape}")
            print(f"  Description: {source_images_container.images[source_image_name].description}")
        else:
            print("❌ No Alexa568 source images found")
        
        # 2. Extract kymograph data from time series
        # Look for TimeSeries line scan data in acquisition
        acquisition_keys = list(nwbfile.acquisition.keys())
        fluo4_timeseries = [key for key in acquisition_keys if 'TimeSeriesLineScanRawFluo4' in key]
        
        if fluo4_timeseries:
            kymograph_name = fluo4_timeseries[0]  # Take the first Fluo4 time series
            kymograph_data = nwbfile.acquisition[kymograph_name].data[:]
            print(f"✓ Extracted kymograph from TimeSeries: {kymograph_name}")
            print(f"  Shape: {kymograph_data.shape}")
            print(f"  Description: {nwbfile.acquisition[kymograph_name].description}")
        else:
            # Fallback: try to get from ROI response series in ophys module
            print("No TimeSeries kymograph found, checking ophys module...")
            if 'ophys' in nwbfile.processing:
                ophys_module = nwbfile.processing['ophys']
                fluorescence = ophys_module.data_interfaces['Fluorescence']
                
                # Look for Fluo4 ROI response series
                fluo4_rois = [name for name in fluorescence.roi_response_series.keys() 
                             if 'Fluo4' in name]
                
                if fluo4_rois:
                    kymograph_name = fluo4_rois[0]  # Take the first Fluo4 ROI
                    roi_data = fluorescence.roi_response_series[kymograph_name]
                    kymograph_data = roi_data.data[:]
                    print(f"✓ Extracted kymograph from ROI: {kymograph_name}")
                    print(f"  Shape: {kymograph_data.shape}")
                    print(f"  Description: {roi_data.description}")
                    
                    # For ROI data, we need to reshape if it's 1D
                    if len(kymograph_data.shape) == 1:
                        # Assume it's a flattened time series, try to infer spatial dimension
                        # This is a simplified approach - in practice you'd need more metadata
                        time_points = len(kymograph_data)
                        spatial_points = 20  # Default assumption based on previous analysis
                        if time_points % spatial_points == 0:
                            temporal_points = time_points // spatial_points
                            kymograph_data = kymograph_data.reshape(temporal_points, spatial_points)
                            print(f"  Reshaped to: {kymograph_data.shape} (time x space)")
                        else:
                            print(f"  Keeping as 1D trace: {kymograph_data.shape}")
        
        # 3. Extract line scan position from ROI plane segmentation
        if 'ophys' in nwbfile.processing:
            ophys_module = nwbfile.processing['ophys']
            image_seg = ophys_module.data_interfaces['ImageSegmentation']
            
            # Find plane segmentations
            plane_seg_names = list(image_seg.plane_segmentations.keys())
            print(f"Available plane segmentations: {plane_seg_names}")
            
            if plane_seg_names:
                # Use the first plane segmentation that matches our data
                plane_seg_name = plane_seg_names[0]
                plane_seg = image_seg.plane_segmentations[plane_seg_name]
                print(f"✓ Found plane segmentation: {plane_seg_name} with {len(plane_seg)} ROIs")
                
                # Extract coordinates from first ROI (line scan region)
                if len(plane_seg) > 0:
                    first_roi = plane_seg[0]
                    pixel_mask = first_roi['pixel_mask'].iloc[0]
                    
                    # Handle different pixel mask formats
                    if hasattr(pixel_mask, 'dtype') and pixel_mask.dtype.names:
                        # Structured array with named fields
                        if 'x' in pixel_mask.dtype.names and 'y' in pixel_mask.dtype.names:
                            x_coords = pixel_mask['x']
                            y_coords = pixel_mask['y']
                            weights = pixel_mask['weight'] if 'weight' in pixel_mask.dtype.names else np.ones(len(x_coords))
                        else:
                            print(f"  Unexpected pixel mask structure: {pixel_mask.dtype.names}")
                            # Try to interpret as coordinate pairs
                            x_coords = pixel_mask[1::2] if len(pixel_mask) % 2 == 0 else pixel_mask[::2]
                            y_coords = pixel_mask[::2] if len(pixel_mask) % 2 == 0 else pixel_mask[1::2]
                            weights = np.ones(len(x_coords))
                    else:
                        # Assume it's a flat array of coordinates or indices
                        if len(pixel_mask) >= 2:
                            # Try to interpret as alternating x,y coordinates
                            if len(pixel_mask) % 2 == 0:
                                x_coords = pixel_mask[::2]
                                y_coords = pixel_mask[1::2]
                            else:
                                # Or as indices into a flattened image
                                if source_image is not None and pixel_mask.max() < source_image.size:
                                    y_coords = pixel_mask // source_image.shape[1]
                                    x_coords = pixel_mask % source_image.shape[1]
                                else:
                                    # Default fallback: create a simple horizontal line
                                    x_coords = np.arange(len(pixel_mask))
                                    y_coords = np.full(len(pixel_mask), source_image.shape[0]//2 if source_image is not None else 64)
                            weights = np.ones(len(x_coords))
                        else:
                            print("  Insufficient pixel mask data")
                            x_coords = y_coords = weights = None
                    
                    if x_coords is not None and y_coords is not None:
                        line_scan_coords = {
                            'x': x_coords,
                            'y': y_coords, 
                            'weights': weights
                        }
                        
                        print(f"✓ Extracted line scan coordinates:")
                        print(f"  X range: {x_coords.min()} to {x_coords.max()}")
                        print(f"  Y range: {y_coords.min()} to {y_coords.max()}")
                        print(f"  Number of pixels: {len(x_coords)}")
                        
                        # Determine line orientation
                        x_range = x_coords.max() - x_coords.min()
                        y_range = y_coords.max() - y_coords.min()
                        
                        if y_range > x_range:
                            orientation = "vertical"
                            position = f"X = {x_coords[0]}"
                            scan_length = y_range + 1
                        else:
                            orientation = "horizontal" 
                            position = f"Y = {y_coords[0]}"
                            scan_length = x_range + 1
                            
                        print(f"  Line orientation: {orientation}")
                        print(f"  Line position: {position}")
                        print(f"  Line length: {scan_length} pixels")
                    else:
                        print("  Could not extract valid coordinates from pixel mask")
        
        # If we still don't have line scan coordinates but have a source image, create default
        if line_scan_coords is None and source_image is not None:
            print("Creating default line scan coordinates...")
            img_height, img_width = source_image.shape
            # Create a horizontal line in the center
            line_length = min(img_width - 20, 20)  # Default to 20 pixels or image width - 20
            start_x = (img_width - line_length) // 2
            y_pos = img_height // 2
            
            x_coords = np.arange(start_x, start_x + line_length)
            y_coords = np.full(line_length, y_pos)
            
            line_scan_coords = {
                'x': x_coords,
                'y': y_coords,
                'weights': np.ones(line_length)
            }
            orientation = "horizontal"
            print(f"✓ Created default horizontal line scan at Y={y_pos}, X={start_x} to {start_x + line_length}")
        
        # Store metadata
        metadata = {
            'session_id': nwbfile.session_id,
            'session_description': nwbfile.session_description,
            'source_image_name': source_image_name if source_image is not None else 'None',
            'kymograph_name': kymograph_name if kymograph_data is not None else 'None',
            'line_orientation': orientation if line_scan_coords else 'unknown',
            'temporal_points': kymograph_data.shape[0] if kymograph_data is not None and len(kymograph_data.shape) >= 1 else 0,
            'spatial_points': kymograph_data.shape[1] if kymograph_data is not None and len(kymograph_data.shape) >= 2 else (len(kymograph_data) if kymograph_data is not None else 0)
        }

# Verify all data was extracted successfully
print(f"\\n=== Extraction Results ===")
print(f"Source image: {'✓' if source_image is not None else '❌'}")
print(f"Kymograph data: {'✓' if kymograph_data is not None else '❌'}")
print(f"Line scan coordinates: {'✓' if line_scan_coords is not None else '❌'}")

if source_image is None or kymograph_data is None or line_scan_coords is None:
    print("\\n⚠️  WARNING: Some data could not be extracted from NWB file")
    print("The visualization will show available data and use defaults for missing components")
else:
    print("\\n✅ All required data successfully extracted!")

print(f"\\n=== Extraction Summary ===")
if source_image is not None:
    print(f"✓ Source image shape: {source_image.shape}")
if kymograph_data is not None:
    print(f"✓ Kymograph shape: {kymograph_data.shape}")
if line_scan_coords is not None:
    print(f"✓ Line scan: {len(line_scan_coords['x'])} pixels, {metadata.get('line_orientation', 'unknown')} orientation")
print(f"✓ Session: {metadata.get('session_id', 'Unknown')}")

# Create visualization regardless of what data is available
if source_image is not None and line_scan_coords is not None:
    # Create the line scan overlay visualization
    plt.figure(figsize=(16, 10))

    # Main source image with line scan overlay
    plt.subplot(2, 2, 1)
    plt.imshow(source_image, cmap='gray', aspect='equal', origin='upper')
    plt.title(f"Source Image with Line Scan Overlay\\n{metadata.get('source_image_name', 'Unknown')}")
    plt.xlabel("X position (pixels)")
    plt.ylabel("Y position (pixels)")

    # Draw the line scan using actual coordinates
    line_x = line_scan_coords['x']
    line_y = line_scan_coords['y']
    plt.plot(line_x, line_y, 'cyan', linewidth=3, alpha=0.8, label='Line scan')

    # Mark start and end of line scan
    plt.scatter(line_x[0], line_y[0], color='green', s=100, marker='o', 
               edgecolor='white', linewidth=2, label='Start', zorder=5)
    plt.scatter(line_x[-1], line_y[-1], color='red', s=100, marker='s', 
               edgecolor='white', linewidth=2, label='End', zorder=5)

    plt.legend()
    plt.colorbar(label='Fluorescence intensity')

    # Zoomed view of the line scan region
    plt.subplot(2, 2, 2)
    margin = 20
    x_min, x_max = max(0, line_x.min() - margin), min(source_image.shape[1], line_x.max() + margin)
    y_min, y_max = max(0, line_y.min() - margin), min(source_image.shape[0], line_y.max() + margin)

    zoomed_image = source_image[y_min:y_max, x_min:x_max]
    plt.imshow(zoomed_image, cmap='gray', aspect='equal', origin='upper',
               extent=[x_min, x_max, y_max, y_min])
    plt.title("Zoomed Line Scan Region")
    plt.xlabel("X position (pixels)")
    plt.ylabel("Y position (pixels)")

    plt.plot(line_x, line_y, 'cyan', linewidth=4, alpha=0.9)
    plt.scatter(line_x[0], line_y[0], color='green', s=150, marker='o', 
               edgecolor='white', linewidth=2, zorder=5)
    plt.scatter(line_x[-1], line_y[-1], color='red', s=150, marker='s', 
               edgecolor='white', linewidth=2, zorder=5)
    plt.colorbar(label='Fluorescence intensity')

    # Show kymograph data if available
    if kymograph_data is not None:
        if len(kymograph_data.shape) == 2:
            plt.subplot(2, 1, 2)
            plt.imshow(kymograph_data.T, aspect='auto', cmap='viridis', origin='lower',
                       extent=[0, metadata['temporal_points'], 0, metadata['spatial_points']])
            plt.title(f"Calcium Kymograph\\n{metadata.get('kymograph_name', 'Unknown')}")
            plt.xlabel("Time (line scan number)")
            plt.ylabel("Position along line scan (pixels)")
            plt.colorbar(label='Fluorescence intensity')
        else:
            # 1D trace
            plt.subplot(2, 1, 2)
            plt.plot(kymograph_data, 'b-', linewidth=1.5)
            plt.title(f"Fluorescence Trace\\n{metadata.get('kymograph_name', 'Unknown')}")
            plt.xlabel("Time points")
            plt.ylabel("Fluorescence intensity")
            plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\\n=== Line Scan Analysis ===")
    print(f"Line scan coordinates extracted from ROI pixel mask:")
    print(f"  Start: ({line_x[0]}, {line_y[0]})")
    print(f"  End: ({line_x[-1]}, {line_y[-1]})")
    print(f"  Length: {len(line_x)} pixels")
    print(f"  Orientation: {metadata.get('line_orientation', 'unknown')}")

    if kymograph_data is not None:
        print(f"\\nKymograph dimensions:")
        print(f"  Shape: {kymograph_data.shape}")
        print(f"  Intensity range: {kymograph_data.min():.1f} to {kymograph_data.max():.1f}")
else:
    print("\\n⚠️  Cannot create full visualization - missing source image or line scan coordinates")
    if kymograph_data is not None:
        plt.figure(figsize=(12, 6))
        if len(kymograph_data.shape) == 2:
            plt.imshow(kymograph_data.T, aspect='auto', cmap='viridis')
            plt.title("Kymograph Data")
            plt.xlabel("Time")
            plt.ylabel("Space")
        else:
            plt.plot(kymograph_data)
            plt.title("Fluorescence Trace")
            plt.xlabel("Time points")
            plt.ylabel("Intensity")
        plt.colorbar()
        plt.show()

## Kymograph Analysis and Visualization

The kymograph data has been successfully extracted from the NWB file and shows the actual line scan measurements from the dendritic excitability experiment.

In [ ]:
# Advanced Analysis: Extract Mean Fluorescence Trace and Detect Events

# Extract mean fluorescence trace from the kymograph (equivalent to LineProfileData.csv)
mean_trace = kymograph_data.mean(axis=1)
time_points = np.arange(len(mean_trace))

# Calculate ΔF/F for calcium analysis
from scipy.ndimage import gaussian_filter1d

# Smooth the trace for baseline estimation
smoothed_trace = gaussian_filter1d(mean_trace, sigma=5)

# Calculate baseline (20th percentile of smoothed data)
baseline = np.percentile(smoothed_trace, 20)

# Calculate ΔF/F
delta_f_over_f = (mean_trace - baseline) / baseline

# Detect calcium events using peak detection
from scipy.signal import find_peaks

# Use adaptive threshold based on signal noise
noise_std = np.std(delta_f_over_f[:100])  # Estimate noise from first 100 points
threshold = 3 * noise_std

peaks, properties = find_peaks(delta_f_over_f, 
                              height=threshold,
                              distance=20,  # Minimum distance between peaks (time points)
                              width=5)      # Minimum event width

# Create comprehensive analysis plot
plt.figure(figsize=(16, 12))

# Raw trace analysis
plt.subplot(4, 1, 1)
plt.plot(time_points, mean_trace, 'blue', linewidth=1.5, alpha=0.8, label='Raw mean trace')
plt.plot(time_points, smoothed_trace, 'red', linewidth=2, label='Smoothed')
plt.axhline(baseline, color='gray', linestyle='--', alpha=0.7, label=f'Baseline ({baseline:.1f})')
plt.title(f"Mean Fluorescence Trace Analysis\\n{metadata['session_id']}")
plt.ylabel("Fluorescence (a.u.)")
plt.legend()
plt.grid(True, alpha=0.3)

# ΔF/F trace with events
plt.subplot(4, 1, 2)
plt.plot(time_points, delta_f_over_f, 'green', linewidth=2, label='ΔF/F')
plt.axhline(threshold, color='red', linestyle='--', alpha=0.7, label=f'Threshold ({threshold:.3f})')
plt.scatter(peaks, delta_f_over_f[peaks], color='red', s=100, zorder=5, 
           label=f'Events ({len(peaks)})', marker='^')
plt.title("Calcium Event Detection")
plt.ylabel("ΔF/F")
plt.legend()
plt.grid(True, alpha=0.3)

# Event characteristics
plt.subplot(4, 1, 3)
if len(peaks) > 0:
    amplitudes = delta_f_over_f[peaks]
    widths = properties['widths']
    
    # Create scatter plot colored by event amplitude
    scatter = plt.scatter(peaks, amplitudes, c=widths, s=120, cmap='plasma', 
                         alpha=0.8, edgecolor='black', linewidth=1)
    plt.colorbar(scatter, label='Event width (time points)')
    
    # Annotate events
    for i, (peak, amp, width) in enumerate(zip(peaks, amplitudes, widths)):
        plt.annotate(f'E{i+1}\\n{amp:.3f}', (peak, amp), 
                    xytext=(0, 15), textcoords='offset points',
                    ha='center', fontsize=9,
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    plt.title(f"Event Characteristics (n={len(peaks)})")
    plt.ylabel("Event amplitude (ΔF/F)")
else:
    plt.text(0.5, 0.5, 'No calcium events detected', 
             transform=plt.gca().transAxes, ha='center', va='center', 
             fontsize=16, style='italic')
    plt.title("Event Characteristics")
    plt.ylabel("Event amplitude (ΔF/F)")

plt.grid(True, alpha=0.3)

# Spatial analysis: kymograph with events marked
plt.subplot(4, 1, 4)
plt.imshow(kymograph_data.T, aspect='auto', cmap='viridis', origin='lower',
           extent=[0, len(mean_trace), 0, kymograph_data.shape[1]])

# Mark detected events as vertical lines
for peak in peaks:
    plt.axvline(peak, color='red', linestyle='--', alpha=0.8, linewidth=2)

plt.title("Kymograph with Detected Events")
plt.xlabel("Time (line scan number)")
plt.ylabel("Position along line scan (pixels)")
plt.colorbar(label='Fluorescence intensity')

plt.tight_layout()
plt.show()

# Print comprehensive analysis results
print(f"=== Calcium Event Analysis Results ===")
print(f"Session: {metadata['session_id']}")
print(f"Kymograph dimensions: {kymograph_data.shape[0]} time points × {kymograph_data.shape[1]} spatial pixels")
print(f"Line scan: {metadata['line_orientation']} orientation, {len(line_scan_coords['x'])} pixels")

print(f"\\nTrace analysis:")
print(f"  Baseline fluorescence: {baseline:.1f}")
print(f"  Mean intensity: {mean_trace.mean():.1f} ± {mean_trace.std():.1f}")
print(f"  ΔF/F range: {delta_f_over_f.min():.3f} to {delta_f_over_f.max():.3f}")
print(f"  Detection threshold: {threshold:.4f}")

print(f"\\nCalcium events detected: {len(peaks)}")
if len(peaks) > 0:
    amplitudes = delta_f_over_f[peaks]
    widths = properties['widths']
    
    print(f"  Event times: {peaks.tolist()}")
    print(f"  Event amplitudes (ΔF/F): {[f'{amp:.3f}' for amp in amplitudes]}")
    print(f"  Event widths (time points): {[f'{w:.1f}' for w in widths]}")
    print(f"  Mean amplitude: {np.mean(amplitudes):.3f} ± {np.std(amplitudes):.3f}")
    print(f"  Mean width: {np.mean(widths):.1f} ± {np.std(widths):.1f} time points")
    print(f"  Event frequency: {len(peaks) / len(mean_trace) * 1000:.2f} events per 1000 time points")

print(f"\\nData extracted from NWB file:")
print(f"  Source image: {metadata['source_image_name']}")
print(f"  Kymograph: {metadata['kymograph_name']}")
print(f"  Line scan position: ROI pixel mask coordinates")

## Advanced Analysis: Detect Calcium Events

Let's add some analysis to detect and characterize calcium transients in the fluorescence trace.

## Summary

This notebook demonstrates how to:

1. **Load NWB files** from the dendritic excitability experiments
2. **Extract fluorescence imaging data** (or create synthetic data for demonstration)
3. **Generate kymographs** showing spatiotemporal fluorescence patterns
4. **Define analysis regions** similar to the original caliper-based selection
5. **Extract mean fluorescence traces** equivalent to LineProfileData.csv
6. **Detect and analyze calcium events** using signal processing techniques

The analysis reproduces the key visualizations from the original code:
- **Kymograph visualization** with analysis region markers
- **Mean fluorescence trace** over time
- **Event detection** and characterization

This framework can be adapted for real NWB data containing fluorescence imaging from dendritic excitability experiments.